In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [57]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [58]:
%run -i ../../core.py

In [59]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [60]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)



In [61]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<0.01*X.size()[1]*(self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [62]:
# prover calculates settings, send to verifier

theory_output = torch.mean(data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<0.01*X.size()[1]*(self.w), self.w)
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(49.9700)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[0,9594],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":606,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [63]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.393484115600586 seconds
Theory output:  tensor(49.9700)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.96875
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[17236403609878300513, 8975289109583201277, 11033320042856086930, 2298871365537728986], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4072474832971751158, 13621019950687641747, 8349885009801153069, 3312433242047143324]]], 'proof': '149c41cca09c3f6c5d46f5fcd4f53fc599e6d5f84a4c7d879dc70a3e480a66c91cbcd3059d8f1f9b3347b48233e4cbb37e06b8e4e1408c571c638407cfb55b142d7674ae436f40cdfc962ad9ea1fb5c9fa10a6fd940a54339b4a15b84e9435371fec920b8c899cac867c966ac871ce8bce63e62172425c90ae551ce4fea7dd67048517a8a28a003f84ffdf2dd5b5c90d0fdcd737344c9cd55bdbab313f3b9c6710057dbbddd2709802467be4d00e774ca224e9fc3897cb3b83aecbc0f0323cc6002c0809f2768a01215bf8201bb1f30bf12ea80552c02d6007f30904deb7827e02d00d185e068de676ada1e3ded9886d53a844ffde0462ae9c87dad2ae397ee01c33aa976076731e271897651fe8f470048d665e95717b95ba711bb37d24364e12b9842ade6d9f519ac24aa833902fd278fd3c140098b76108301309e387c72b1ed5e5a84ec1e15ed2bdc1a2fc78efbc3a04a14f0bfb0a1108fa64717ab2a4

In [64]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17236403609878300513, 8975289109583201277, 11033320042856086930, 2298871365537728986], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4072474832971751158, 13621019950687641747, 8349885009801153069, 3312433242047143324]]]
proof boolean:  1.0
proof result 1 : 49.96875
verified
